# KO-T5를 이용한 문장 순서 예측

이 노트북에서는 nlpHyperion/ko-t5-base 모델을 사용하여 문장 순서 예측을 구현합니다. 다음과 같은 과정을 수행합니다:

1. 데이터 로드 및 전처리
2. 모델 및 학습 설정
3. Early stopping과 체크포인팅을 적용한 학습
4. 모델 평가
5. 하이퍼파라미터 튜닝

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.optim import AdamW
from tqdm.notebook import tqdm
import os

from t5_utils import SentenceOrderPredictor, compute_accuracy

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')

# 결과 저장을 위한 디렉토리 생성
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('history', exist_ok=True)
os.makedirs('grid_search_results', exist_ok=True)

## 1. 데이터 로드 및 준비

In [ ]:
# 전처리된 데이터 로드
df = pd.read_csv('../data/cleaned_seq2seq.csv')

# 데이터 확인
print("데이터셋 크기:", len(df))
print("\n샘플 데이터:")
df.head(5)

In [ ]:
# ko-t5-base 모델 초기화
predictor = SentenceOrderPredictor(
    model_name="nlpHyperion/ko-t5-base",
    use_auto_classes=True  # AutoTokenizer/AutoModel 사용
)

# 데이터셋 준비 (9:1 비율로 분할)
train_dataset, val_dataset = predictor.prepare_data(df)

# 데이터로더 생성
batch_size = 8  # GPU 메모리에 따라 조정 가능
train_loader, val_loader = predictor.create_dataloaders(train_dataset, val_dataset, batch_size=batch_size)

print(f"학습 데이터 크기: {len(train_dataset)}")
print(f"검증 데이터 크기: {len(val_dataset)}")

## 2. 학습 설정

In [ ]:
# 학습 파라미터
num_epochs = 10
learning_rate = 5e-5
patience = 3  # Early stopping 인내심
min_delta = 1e-4  # 최소 개선 기준

# 옵티마이저 설정
optimizer = AdamW(predictor.model.parameters(), lr=learning_rate)

# 학습 히스토리 초기화
history = {
    'train_loss': [],
    'val_loss': [],
    'val_accuracy': []
}

## 3. Early Stopping을 적용한 학습

In [ ]:
def validate(model, val_loader, device, tokenizer):
    """검증 데이터에 대한 모델 평가"""
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            total_loss += loss.item()
            
            # 예측 생성
            predictions = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=8,
                num_beams=4,
                no_repeat_ngram_size=4  # 숫자 중복 방지
            )
            
            # 예측과 레이블을 순서 시퀀스로 변환
            pred_texts = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]
            label_texts = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
            
            try:
                # 예측과 레이블을 정수 리스트로 변환
                pred_orders = [list(map(int, text.split())) for text in pred_texts]
                label_orders = [list(map(int, text.split())) for text in label_texts]
                
                # 정확도 계산
                correct = sum(compute_accuracy(pred, label) for pred, label in zip(pred_orders, label_orders))
                total_correct += correct
                total_samples += len(input_ids)
            except ValueError as e:
                print(f"Warning: 잘못된 예측 형식 발견 - {e}")
                print(f"Predictions: {pred_texts}")
                print(f"Labels: {label_texts}")
                continue
    
    avg_loss = total_loss / len(val_loader)  # 배치 수로 나누기
    accuracy = total_correct / total_samples if total_samples > 0 else 0
    
    return avg_loss, accuracy

In [ ]:
# 학습 루프
best_val_loss = float('inf')
best_val_accuracy = 0
best_epoch = -1
patience_counter = 0

for epoch in range(num_epochs):
    # 학습 단계
    predictor.model.train()
    total_train_loss = 0
    
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(predictor.device)
        attention_mask = batch['attention_mask'].to(predictor.device)
        labels = batch['labels'].to(predictor.device)
        
        outputs = predictor.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_loader)
    
    # 검증
    val_loss, val_accuracy = validate(predictor.model, val_loader, predictor.device, predictor.tokenizer)
    
    # 히스토리 업데이트
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_accuracy)
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {avg_train_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    
    # 검증 손실이 개선된 경우 체크포인트 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        predictor.save_checkpoint(
            epoch,
            predictor.model,
            optimizer,
            val_loss,
            f'checkpoints/ko_t5_best_model.pt'
        )
    else:
        patience_counter += 1
        
    # Early stopping 체크
    if patience_counter >= patience:
        print(f'Epoch {epoch+1} 이후 Early stopping 발동')
        break

# 학습 히스토리 저장
predictor.save_history(history, 'history/ko_t5_history.json')

## 4. 학습 결과 시각화

In [ ]:
plt.figure(figsize=(12, 4))

# Loss 그래프
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train 및 Validation Loss')
plt.legend()

# Accuracy 그래프
plt.subplot(1, 2, 2)
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 5. 하이퍼파라미터 튜닝

In [ ]:
def train_with_params(learning_rate, batch_size, max_epochs, train_dataset, val_dataset, model_name="nlpHyperion/ko-t5-base", use_auto_classes=False):
    """주어진 하이퍼파라미터로 모델 학습"""
    # 모델과 옵티마이저 초기화
    predictor = SentenceOrderPredictor(model_name=model_name, use_auto_classes=use_auto_classes)
    optimizer = AdamW(predictor.model.parameters(), lr=learning_rate)
    
    # 데이터로더 생성
    train_loader, val_loader = predictor.create_dataloaders(train_dataset, val_dataset, batch_size=batch_size)
    
    best_val_loss = float('inf')
    best_val_accuracy = 0
    patience_counter = 0
    patience = 3
    
    for epoch in range(max_epochs):
        # 학습
        predictor.model.train()
        total_train_loss = 0
        
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{max_epochs} - Training'):
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(predictor.device)
            attention_mask = batch['attention_mask'].to(predictor.device)
            labels = batch['labels'].to(predictor.device)
            
            outputs = predictor.model(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    labels=labels)
            
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_loader)
        
        # Validation
        val_loss, val_accuracy = validate(predictor.model, val_loader, predictor.device, predictor.tokenizer)
        
        print(f'Epoch {epoch+1}/{max_epochs}')
        print(f'Train Loss: {avg_train_loss:.4f}')
        print(f'Validation Loss: {val_loss:.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_accuracy = val_accuracy
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:  # Early stopping
            print(f'Early stopping at epoch {epoch+1}')
            break
    
    return best_val_loss, best_val_accuracy

In [ ]:
def run_grid_search(train_dataset, val_dataset, model_name="paust/pko-t5-base", use_auto_classes=False):
    """그리드 서치 수행"""
    # 그리드 서치 파라미터
    param_grid = {
        'learning_rate': [1e-5, 2e-5, 5e-5],
        'batch_size': [4, 8, 16],
        'max_epochs': [5, 10]
    }

    # 결과 저장용 리스트
    results = []
    best_params = None
    best_accuracy = 0

    # 총 시도 횟수 계산
    total_trials = len(param_grid['learning_rate']) * len(param_grid['batch_size']) * len(param_grid['max_epochs'])
    trial_count = 0

    for lr in param_grid['learning_rate']:
        for bs in param_grid['batch_size']:
            for epochs in param_grid['max_epochs']:
                trial_count += 1
                print(f'\n[Trial {trial_count}/{total_trials}]')
                print(f'Parameters: Learning Rate={lr}, Batch Size={bs}, Max Epochs={epochs}')
                
                val_loss, val_acc = train_with_params(
                    learning_rate=lr,
                    batch_size=bs,
                    max_epochs=epochs,
                    train_dataset=train_dataset,
                    val_dataset=val_dataset,
                    model_name=model_name,
                    use_auto_classes=use_auto_classes
                )
                
                results.append({
                    'learning_rate': lr,
                    'batch_size': bs,
                    'max_epochs': epochs,
                    'val_loss': val_loss,
                    'val_accuracy': val_acc
                })
                
                # 최고 성능 업데이트
                if val_acc > best_accuracy:
                    best_accuracy = val_acc
                    best_params = {
                        'learning_rate': lr,
                        'batch_size': bs,
                        'max_epochs': epochs
                    }
                
                print(f'Results: Val Loss={val_loss:.4f}, Val Accuracy={val_acc:.4f}')

    # 결과를 DataFrame으로 변환
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('val_accuracy', ascending=False)
    
    # 결과 저장
    results_df.to_csv(f'grid_search_results/ko_t5_results.csv', index=False)
    
    print('\n=== 그리드 서치 완료 ===')
    print('\n최적의 하이퍼파라미터:')
    print(f'Learning Rate: {best_params["learning_rate"]}')
    print(f'Batch Size: {best_params["batch_size"]}')
    print(f'Max Epochs: {best_params["max_epochs"]}')
    print(f'Best Validation Accuracy: {best_accuracy:.4f}')
    
    return results_df, best_params

In [ ]:
# 그리드 서치 실행
results_df, best_params = run_grid_search(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    model_name="nlpHyperion/ko-t5-base",
    use_auto_classes=False
)

# 상위 5개 결과 표시
results_df.head()

## 6. 최적 모델로 예측 테스트

In [ ]:
# 최적 모델 불러오기
predictor.load_checkpoint('checkpoints/ko_t5_best_model.pt')

# 테스트할 텍스트
test_text = df['input_text'].iloc[0]  # 첫 번째 샘플로 테스트
true_order = df['target_text'].iloc[0]

# 예측
predicted_order = predictor.predict_order(test_text)

print("입력 텍스트:")
print(test_text)
print("\n예측된 순서:", predicted_order)
print("실제 순서:", true_order)
print("정확도:", compute_accuracy(predicted_order, true_order))